In [1]:
import importlib
import types
from eth2spec.config.config_util import prepare_config
from eth2spec.utils.ssz.ssz_impl import hash_tree_root

import os, sys
# Load current beacon runner specs (v1.1.0-alpha.3)
sys.path.insert(1, os.path.realpath(os.path.pardir))

import beaconrunner as br

prepare_config(".", "fast.yaml")

br.reload_package(br)

In [4]:
def extract_state(s):
    validators = s["network"].validators
    validator = validators[1]
    head = br.specs.get_head(validator.store)
    current_state = validator.store.block_states[head].copy()
    return current_state

def current_slot(params, step, sL, s, _input):
    return ("current_slot", s["network"].validators[0].data.slot)

def total_balance_asap(params, step, sL, s, _input):
    validators = s["network"].validators
    current_state = extract_state(s)
    current_epoch = br.specs.get_current_epoch(current_state)
    asap_indices = [i for i, v in enumerate(validators) if v.validator_behaviour == "asap"]
    asap_balances = [b for i, b in enumerate(current_state.balances) if i in asap_indices]
    return ("total_balance_asap", sum(asap_balances))

def total_balance_malicious(params, step, sL, s, _input):
    validators = s["network"].validators
    current_state = extract_state(s)
    current_epoch = br.specs.get_current_epoch(current_state)
    malicious_indices = [i for i, v in enumerate(validators) if v.validator_behaviour == "malicious"]
    malicious_balances = [b for i, b in enumerate(current_state.balances) if i in malicious_indices]
    return ("total_balance_malicious", sum(malicious_balances))

def get_base_reward(params, step, sL, s, _input):
    current_state = extract_state(s)
    base_reward = br.specs.get_base_reward(current_state, 0)
    return ("base_reward", base_reward)

def get_block_proposer(params, step, sL, s, _input):
    current_state = extract_state(s)
    block_proposer = [v.validator_index for v in s["network"].validators if v.data.current_proposer_duties[s["current_slot"] % br.specs.SLOTS_PER_EPOCH]][0]
    return ("block_proposer", block_proposer)

def get_block_proposer_balance(params, step, sL, s, _input):
    current_state = extract_state(s)
    block_proposer_balance = current_state.balances[s["block_proposer"]]
    return ("block_proposer_balance", block_proposer_balance)

def get_sync_committee(params, step, sL, s, _input):
    current_state = extract_state(s)
    current_epoch = br.specs.get_current_epoch(current_state)
    sync_committee = current_state.current_sync_committee
    sync_committee = br.specs.get_sync_committee_indices(current_state, current_epoch)
    return ("sync_committee", sync_committee)

def get_head(params, step, sL, s, _input):
    validators = s["network"].validators
    validator = validators[0]
    head = br.specs.get_head(validator.store).hex()[0:6]
    return ("head", head)

def get_malicious_head(params, step, sL, s, _input):
    malicious_head = s["malicious_data"].latest_malicious_head
    return ("malicious_head", malicious_head)

def get_current_validator_state(params, step, sL, s, _input):
    current_state = extract_state(s)
    current_validator_state = []
    for v in s["network"].validators:
        current_validator_state += [{
            "slot": v.data.slot,
            "validator_index": v.validator_index,
            "balance": current_state.balances[v.validator_index],
            "block_proposer": 1 if s["block_proposer"] == v.validator_index else 0,
            "attester": 1 if v.data.current_attest_slot == v.data.slot else 0,
            "sync_committee": len(v.data.current_sync_committee),
        }]
    return ("current_validator_state", current_validator_state)

observers = {
    "current_slot": current_slot,
    "total_balance_asap": total_balance_asap,
    "total_balance_malicious": total_balance_malicious,
    "base_reward": get_base_reward,
    "block_proposer": get_block_proposer,
    "block_proposer_balance": get_block_proposer_balance,
#     "sync_committee": get_sync_committee,
    "head": get_head,
    "malicious_head": get_malicious_head,
    "current_validator_state": get_current_validator_state,
}

In [ ]:
from random import sample
from beaconrunner.validators.ASAPValidator import ASAPValidator
from beaconrunner.validators.MaliciousValidator import MaliciousValidator

def simulate_once(network_sets, num_run, num_validators, network_update_rate):
    
    num_malicious = int((3 * num_validators) / 10)

    # We sample the position on the p2p network of prudent validators randomly
    maliciousset = set(sample(range(num_validators), num_malicious))

    validators = []

    # Initiate validators
    for i in range(num_validators):
        if i in maliciousset:
            new_validator = MaliciousValidator(i)
        else:
            new_validator = ASAPValidator(i)
        validators.append(new_validator)
    
    # Create a genesis state
    genesis_state = br.simulator.get_genesis_state(validators)
    
    # Validators load the state
    [v.load_state(genesis_state.copy()) for v in validators]

    br.simulator.skip_genesis_block(validators)

    network = br.network.Network(validators = validators, sets=network_sets)

    parameters = br.simulator.SimulationParameters({
        "num_epochs": 20,
        "num_run": num_run,
        "frequency": 1,
        "network_update_rate": network_update_rate,
    })

    return br.simulator.simulate(network, parameters, observers)

In [ ]:
%%capture

import pandas as pd

num_validators = 12

# Create the network peers
set_a = br.network.NetworkSet(validators=list(range(num_validators)))
network_sets = list([set_a])

num_runs = 40
network_update_rate = 0.25

df = pd.concat([simulate_once(network_sets, num_run, num_validators, network_update_rate) for num_run in range(num_runs)]